In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout, LSTM
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

%matplotlib inline


class Config:
    def __init__(self):
        pass


def set_config(config_dict):
    config = Config()
    config.__dict__ = config_dict
    return config


In [2]:
config = set_config({
    'row_start': 7228,
    'row_end': 7288,
    "timesteps": 2,
    "max_batch_size": 13,
    "layer_size": 1,
    "unit_size": 1,
    "dropout": 0,
    "learning_rate": 0.1,
    "max_epochs": 1,
    "time_col": 'tanggal',
    "feature": ['rr']
})

# usecols = [*[config.time_col], *config.feature]


def train_test_split(dataset, time_step=1):
    dataX, dataY = [], []
    # for i in range(len(dataset)-time_step-1):
    for i in range(len(dataset)-time_step):
        dataX.append(dataset[i:(i+time_step), 0])
        dataY.append(dataset[i+time_step, 0])
    return np.array(dataX), np.array(dataY)


In [3]:
dataset = pd.read_csv('../Data/1985-2021.csv').replace(to_replace=[8888, 9999, 2555], value=np.nan)[config.row_start:config.row_end]
dataset.interpolate(inplace=True)

datelist = [datetime.strptime(date, '%Y-%m-%d').date() for date in list(dataset[config.time_col])]

In [4]:
featureset = dataset[config.feature]
vector_featureset = featureset.values

scaller = MinMaxScaler()
vector_featureset_scaled = scaller.fit_transform(vector_featureset)

train_size = int(vector_featureset_scaled.size * 0.9)
trainset, testset = vector_featureset_scaled[0:train_size], vector_featureset_scaled[train_size:vector_featureset_scaled.size]

X_train, y_train = train_test_split(trainset, time_step=config.timesteps)
X_test, y_test = train_test_split(testset, time_step=config.timesteps)
(51, 2)

X_train = np.reshape(X_train, (X_train.shape[0], len(config.feature), X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], len(config.feature), X_test.shape[1]))
# (51, 1, 2)

In [5]:
X_train.shape

(52, 1, 2)

In [6]:
y_train.shape

(52,)

In [7]:
y_pred_loss = list()
y_true_loss = list()


def mean_squared_error(y_true, y_pred):
    y_pred_loss.append(y_pred)
    y_true_loss.append(y_true)
    return K.mean(K.square(y_pred - y_true))


def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# model = Sequential()

# for i in range(0, config.layer_size):
#     model.add(LSTM(
#         units=config.unit_size,
#         return_sequences=False if i == config.layer_size - 1 else True,
#         input_shape=(config.timesteps, 1),
#         go_backwards=True,
#         dropout=config.dropout,
#         weights=[
#             np.float32([[0.5774, 0.5774, 0.5774, 0.5774]]),
#             np.float32([[0.5774, 0.5774, 0.5774, 0.5774]]),
#             np.float32([0, 0, 0, 0])
#         ],
#     ))
# else:
#     # model.compile(optimizer=SGD(learning_rate=config.learning_rate), loss=root_mean_squared_error)
#     print(model.get_weights())
#     model.compile(optimizer=SGD(learning_rate=config.learning_rate), loss='mean_squared_error')
#     model.summary()


model = Sequential()

model.add(LSTM(
    config.unit_size,
    # input_shape=(len(config.feature), config.timesteps),
    batch_input_shape=(13, 1, 2),
    go_backwards=True,
    dropout=config.dropout,
    weights=[
        np.float32([
            [0.5774, 0.5774, 0.5774, 0.5774],
            [0.5774, 0.5774, 0.5774, 0.5774]
        ]),
        np.float32([
            [0.5774, 0.5774, 0.5774, 0.5774]
        ]),
        np.float32(
            [0, 0, 0, 0]
        )
    ],
))

model.compile(
    optimizer=SGD(learning_rate=config.learning_rate),
    loss=mean_squared_error,
    run_eagerly=True
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (13, 1)                   16        
                                                                 
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


In [8]:
batch_loss = list()

class CustomCallback(Callback):
    def on_train_batch_end(self, batch, logs=None):
        batch_loss.append(logs)
        
history = model.fit(X_train, y_train, shuffle=False, epochs=config.max_epochs, verbose=1, batch_size=config.max_batch_size, callbacks=[CustomCallback()])

4/4 [==============================] - 0s 56ms/step - loss: 0.0863


In [9]:
batch_loss

[{'loss': 0.10938253998756409},
 {'loss': 0.07940879464149475},
 {'loss': 0.08572952449321747},
 {'loss': 0.08628234267234802}]

In [10]:
y_pred_loss[0]

<tf.Tensor: shape=(13, 1), dtype=float32, numpy=
array([[0.00377558],
       [0.2109283 ],
       [0.32661963],
       [0.24064206],
       [0.17232989],
       [0.0582256 ],
       [0.04636811],
       [0.0313885 ],
       [0.00316408],
       [0.02445099],
       [0.04007021],
       [0.06351911],
       [0.07628259]], dtype=float32)>

In [11]:
y_true_loss[0]

<tf.Tensor: shape=(13, 1), dtype=float32, numpy=
array([[1.        ],
       [0.52577317],
       [0.6206186 ],
       [0.2371134 ],
       [0.10309278],
       [0.17525773],
       [0.02061856],
       [0.00103093],
       [0.15463917],
       [0.08969072],
       [0.27731958],
       [0.15257733],
       [0.4402062 ]], dtype=float32)>